## Daten aus mysql in mongodb importieren

In [20]:
# import pymongo
# import mysql.connector
# import schedule
# import time

# def load_data():
#     # Verbindung zur MongoDB herstellen
#     mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
#     mongo_db = mongo_client['deine_mongo_db']
#     mongo_collection = mongo_db['deine_mongo_collection']

#     # Verbindung zu MySQL herstellen
#     mysql_connection = mysql.connector.connect(
#         host="localhost",
#         user="dein_mysql_benutzer",
#         password="dein_mysql_passwort",
#         database="deine_mysql_db"
#     )
#     mysql_cursor = mysql_connection.cursor()

#     # MongoDB-Daten abrufen
#     mongo_data = mongo_collection.find()

#     # MySQL-Tabelle erstellen (falls noch nicht vorhanden)
#     mysql_cursor.execute("CREATE TABLE IF NOT EXISTS deine_mysql_tabelle (col1 VARCHAR(255), col2 INT)")

#     # MongoDB-Daten in MySQL einfügen
#     for document in mongo_data:
#         col1_value = document.get("field1", "")
#         col2_value = document.get("field2", 0)
#         mysql_cursor.execute("INSERT INTO deine_mysql_tabelle (col1, col2) VALUES (%s, %s)", (col1_value, col2_value))

#     # Änderungen in der MySQL-Datenbank speichern
#     mysql_connection.commit()

#     # Verbindungen schließen
#     mysql_cursor.close()
#     mysql_connection.close()
#     mongo_client.close()

# # Zeitplan für alle 30 Minuten einrichten
# schedule.every(30).minutes.do(load_data)

# while True:
#     schedule.run_pending()
#     time.sleep(1)


## Every business has a unique business_id and they could just see the reviews of their business

In [4]:
import pymongo
from bson.objectid import ObjectId

# Establish connection to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
# Access the database
db = client['yelp']

# Access the collections
review_collection = db['review']
user_collection = db['user']
business_collection = db['business']
admin_collection = db['admin']

def authenticate_user(admin_username, admin_password):
    existing_admin = admin_collection.find_one({"admin_username": admin_username})

    if existing_admin is None:
        return None, "Wrong username. Please enter the right username and password."

    # Check if the password matches
    if existing_admin['admin_password'] != admin_password:
        return None, "Wrong password. Please enter the right username and password."

    business_id = existing_admin['business_id']

    # Retrieve reviews for the specified business_id
    reviews = review_collection.find({"business_id": business_id})
    return list(reviews)

# Beispielaufruf:
admin_username = input("Admin Username: ")
admin_password = input("Admin Password: ")

results = authenticate_user(admin_username, admin_password)
for result in results:
    print(result)

None
Wrong password. Please enter the right username and password.


## Query abfragung mit mysql

In [22]:
# import mysql.connector

# # Verbindung zur MySQL-Datenbank herstellen
# db = mysql.connector.connect(
#     host="localhost",
#     user="root",  # Changed from admin_username to user
#     password="mysql",  # Changed from admin_password to password
#     database="yelp"
# )

# # Beispielabfrage: Alle Bewertungen für ein bestimmtes Unternehmen in einem bestimmten Zeitraum
# business_name = "Abby Rappoport, LAC, CMQ"
# start_date = "2023-01-01"
# end_date = "2023-12-31"

# cursor = db.cursor()
# query = f"SELECT * FROM review WHERE business_name = '{business_name}' AND date BETWEEN '{start_date}' AND '{end_date}'"
# cursor.execute(query)

# # Ergebnisse anzeigen
# for review in cursor.fetchall():
#     print(review)


C:\Users\Tristan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\mysql\connector\abstracts.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if group is 'connector_python':
C:\Users\Tristan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\mysql\connector\abstracts.py:130: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if group is 'connector_python':


InterfaceError: 2003: Can't connect to MySQL server on 'localhost:3306' (10061 No connection could be made because the target machine actively refused it)

## Query abfragung mit MongoDB

In [23]:
# import pymongo
# from datetime import datetime

# # Verbindung zur MongoDB herstellen
# client = pymongo.MongoClient("mongodb://localhost:27017/")
# db = client['yelp']

# review_collection = db['review']
# user_collection = db['user']
# business_collection = db['business']
# admin_collection = db['admin']

# # Beispielabfrage: Alle Bewertungen für ein bestimmtes Unternehmen in einem bestimmten Zeitraum
# business_name = "Abby Rappoport, LAC, CMQ"
# start_date = datetime(2023, 1, 1)  # Define your start date
# end_date = datetime(2023, 12, 31)  # Define your end date
# min_star = 1  # Minimum star rating
# max_star = 3  # Maximum star rating

# # Query for reviews based on business name, star range, and date range
# reviews = review_collection.find({
#     "business_name": business_name,
#     "stars": {"$gte": min_star, "$lte": max_star},  # Filter reviews based on the star range
#     "date": {"$gte": start_date, "$lte": end_date}  # Filter reviews based on date range
# })

# # Ergebnisse anzeigen
# for review in reviews:
#     print(review)

Query results (with filters):
[]


In [5]:
import pymongo
from datetime import datetime

# Establish connection to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['yelp']

# Access the collections
review_collection = db['review']
user_collection = db['user']
business_collection = db['business']
admin_collection = db['admin']

# ... (previous code remains the same)

def authenticate_user(admin_username, admin_password, start_date, end_date, min_stars, max_stars):
    existing_admin = admin_collection.find_one({"admin_username": admin_username})

    if existing_admin is None:
        return None, "Wrong username. Please enter the right username and password."

    # Check if the password matches
    if existing_admin['admin_password'] != admin_password:
        return None, "Wrong password. Please enter the right username and password."

    business_id = existing_admin['business_id']

    # Format start_date and end_date into strings matching the database format
    formatted_start_date = start_date.strftime("%Y-%m-%d %H:%M:%S")
    formatted_end_date = end_date.strftime("%Y-%m-%d %H:%M:%S")

    # Retrieve reviews for the specified business_id within the time range and star range
    reviews = review_collection.find({
        "business_id": business_id,
        "date": {"$gte": formatted_start_date, "$lte": formatted_end_date},
        "stars": {"$gte": min_stars, "$lte": max_stars}
    })
    return list(reviews), None

# Beispielaufruf:
admin_username = input("Admin Username: ")
admin_password = input("Admin Password: ")

start_year = int(input("Start Year: "))
start_month = int(input("Start Month: "))
start_day = int(input("Start Day: "))
start_hour = int(input("Start Hour: "))
start_minute = int(input("Start Minute: "))
start_second = int(input("Start Second: "))

end_year = int(input("End Year: "))
end_month = int(input("End Month: "))
end_day = int(input("End Day: "))
end_hour = int(input("End Hour: "))
end_minute = int(input("End Minute: "))
end_second = int(input("End Second: "))

# Define the start and end dates for the time range (modify as needed)
start_date = datetime(start_year, start_month, start_day, start_hour, start_minute, start_second)
end_date = datetime(end_year, end_month, end_day, end_hour, end_minute, end_second)

# Define the star rating range (modify as needed)
min_stars = 3
max_stars = 5

results, error = authenticate_user(admin_username, admin_password, start_date, end_date, min_stars, max_stars)
if error:
    print(error)
else:
    for result in results:
        print(result)

{'_id': ObjectId('656b238d8895f3b51c4d3e57'), 'review_id': '9vwYDBVI3ymdqcyJ5WW2Tg', 'user_id': 'e0imecnX_9MtLnS2rUZM-A', 'business_id': 'Pns2l4eNsfO8kk83dixA6A', 'stars': 5, 'useful': 3, 'funny': 2, 'cool': 1, 'text': "I've had acupuncture treatments with Abby over the past few years and I really can't say enough about how great my experiences have been.\n\nShe has successfully treated me for everything from stress to colds to menstrual cramps.  I was amazed that her medical qi gong treatments helped alleviate my chronic back pain.  I didn't think anything could help with that!  \n\nShe is always very clear and upfront about treatments and expectations.   And I love that she always has great advice on herbs, supplements and natural remedies.\n\nShe just relocated to Los Gatos and I'm definitely going to miss having her in San Francisco!", 'date': '2012-05-02 18:07:38'}
{'_id': ObjectId('656b23928895f3b51c4e06d9'), 'review_id': 'OXgg1LdxHDv3CBU5-xi2lA', 'user_id': '_Q0fdLVoTnlNkEypUvNk

In [1]:
import pymongo
import schedule
import time
from datetime import datetime
import redis
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import sys

# Establish connection to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Establish connection to Redis using Docker container host
redis_client = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)

# Access the database
db = client['yelp']

# Access the collections
review_collection = db['review']
business_collection = db['business']
admin_collection = db['admin']

# Storing previous review IDs
previous_review_ids = set()

def email_exists(email):
    existing_admin = admin_collection.find_one({"admin_email": email})
    return existing_admin is not None

def business_exists(business_name):
    existing_business = admin_collection.find_one({"business_name": business_name})
    return existing_business is not None

def send_email(receiver_email, subject, body):
    sender_email = "eunicelimuria@gmail.com"  # Replace with your email
    sender_password = "qtyo xhpj mpuk znjt"   # Replace with your email password

    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = receiver_email
    message["Subject"] = subject

    message.attach(MIMEText(body, "plain"))

    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, receiver_email, message.as_string())

def notify_low_rating_reviews(username, password, threshold_percentage, last_n_reviews, business_id, admin_email):
    if admin_collection.find_one({"admin_username": username})['admin_password'] != password:
        raise ValueError("Wrong password. Please enter the right username and password.")
    
    threshold_percentage = admin_collection.find_one({"admin_username": username})['threshold_percentage']
    last_n_reviews = admin_collection.find_one({"admin_username": username})['last_n_reviews']
    business_id = admin_collection.find_one({"admin_username": username})['business_id']
    business_name = business_collection.find_one({"business_id": business_id})['name']

    current_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    admin = admin_collection.find_one({"business_id": business_id})
    if not admin:
        return "Admin not found"
    
    # Find the last 'n' reviews for the given business_id based on the "date" field
    recent_reviews = list(review_collection.find(
        {"business_id": business_id}
    ).sort([("date", -1)]).limit(last_n_reviews))

    print(f"Recent reviews for Admin {username}:")
    for review in recent_reviews:
        print(f"Review: {review['_id']} - Stars: {review['stars']} - Date: {review['date']}")

    # Get the set of current review IDs
    current_review_ids = {review['_id'] for review in recent_reviews}

    # Check if there are new reviews by comparing review IDs
    new_reviews = current_review_ids - previous_review_ids

    # Update previous_review_ids with the current review IDs
    previous_review_ids.update(current_review_ids)

    admin_name = admin['admin_name']

    if new_reviews:  # If there are new reviews, proceed to send an email
        # Calculate the percentage of reviews with stars 1 or 2 out of the last 'n' reviews
        low_rating_count = sum(1 for review in recent_reviews if review['stars'] in [1, 2])
        total_reviews = len(recent_reviews)
        percentage = (low_rating_count / total_reviews) * 100 if total_reviews > 0 else 0

        # Notify if the threshold percentage is exceeded
        if percentage >= threshold_percentage:
            redis_channel = "admin_negative_reviews_channel"
            redis_message = f"Business: {business_name} received {percentage} negative reviews"
            redis_client.publish(redis_channel, redis_message)

            # Send email notification
            email_subject = "Negative Reviews Notification"
            email_body = f"Dear {admin_name},\n\nYour business '{business_name}' has received {percentage:.2f}% negative reviews of the last {last_n_reviews} reviews.\n\nChecked on {current_date}\n\nSincerely,\nYelp Review System."
            send_email(admin_email, email_subject, email_body)

            return f"Checked on {current_date}. Threshold exceeded: {percentage:.2f}% of the last {last_n_reviews} reviews for '{business_name}' have low ratings."

    return None

def job():
    try:
        admins = admin_collection.find({})
        for admin in admins:
            username = admin['admin_username']
            password = admin['admin_password']
            threshold_percentage = admin['threshold_percentage']
            last_n_reviews = admin['last_n_reviews']
            business_id = admin['business_id']
            admin_email = admin['admin_email']

            result = notify_low_rating_reviews(username, password, threshold_percentage, last_n_reviews, business_id, admin_email)
            if result:
                print(result)
    except ValueError as err:
        print(err)
        sys.exit()

# Schedule the job to run every second (adjust as needed)
schedule.every(1).second.do(job)

# Run the job initially
job()

# Keep the script running
while True:
    schedule.run_pending()
    time.sleep(30)

Recent reviews for Admin target_blvd5255:
Review: 280458dc-3c0b-4910-840b-6aecee6c792c - Stars: 1 - Date: 2023-12-05 08:38:18
Review: 656b23c78895f3b51c54a7fc - Stars: 4 - Date: 2020-12-02 02:50:27
Review: 656b23ab8895f3b51c518f20 - Stars: 3 - Date: 2019-04-13 09:01:50
Review: 656b23d48895f3b51c56143c - Stars: 3 - Date: 2018-12-21 09:20:13
Review: 656b23a58895f3b51c50b6f0 - Stars: 1 - Date: 2018-11-12 04:27:49
Review: 656b23a88895f3b51c511729 - Stars: 5 - Date: 2018-09-20 02:32:01
Review: 656b23ca8895f3b51c54fa21 - Stars: 5 - Date: 2018-09-14 00:38:39
Review: 656b23a28895f3b51c504152 - Stars: 1 - Date: 2017-12-01 19:58:25
Review: 656b23988895f3b51c4ed5d5 - Stars: 4 - Date: 2017-08-14 23:34:39
Review: 656b23858895f3b51c4c34be - Stars: 4 - Date: 2017-02-19 15:11:22
Checked on 2023-12-05 12:53:49. Threshold exceeded: 30.00% of the last 10 reviews for 'Target' have low ratings.
Recent reviews for Admin abby:
Review: 656b239c8895f3b51c4f684d - Stars: 5 - Date: 2015-03-16 03:43:08
Review: 65

KeyboardInterrupt: 